In [2]:
# Import necessary libraries and the encoder module
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import datasets
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.callbacks import CSVLogger

In [3]:
# Import from encoder.py
from encoder import tokenize_and_pad, EncoderCategoryTransformer

In [4]:
# Load GoEmotions dataset
emotion_data = load_dataset("go_emotions", "simplified")
emotion_data = emotion_data.filter(lambda x: len(x["labels"]) == 1)

all_emotions = ["admiration", "amusement", "anger", "annoyance", "approval", "caring", "confusion", "curiosity", "desire",
                "disappointment", "disapproval", "disgust", "embarrassment", "excitement", "fear", "gratitude", "grief", "joy",
                "love", "nervousness", "optimism", "pride", "realization", "relief", "remorse", "sadness", "surprise", "neutral"]

num_label = len(all_emotions)

In [5]:
# Transform labels
def one_hot_encode(labels, num_classes):
    mlb = MultiLabelBinarizer()
    mlb.fit([list(range(num_classes))])
    return mlb.transform(labels)

In [6]:
# Tokenizer for GoEmotions dataset
num_words = 10000
max_length = 120
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<OOV>')
tokenizer.fit_on_texts(emotion_data["train"]["text"])

In [7]:
# Data preparation for Transformer model
X_train_transformer = tokenize_and_pad(emotion_data["train"]["text"], tokenizer, max_length)
X_test_transformer = tokenize_and_pad(emotion_data["test"]["text"], tokenizer, max_length)
Y_train_transformer = one_hot_encode(emotion_data["train"]["labels"], num_label)
Y_test_transformer = one_hot_encode(emotion_data["test"]["labels"], num_label)

In [8]:
# Data preparation for traditional models
vectorizer = CountVectorizer(max_features=num_words)
X_train_traditional = vectorizer.fit_transform(emotion_data["train"]["text"])
Y_train_traditional = one_hot_encode(emotion_data["train"]["labels"], num_label)
X_test_traditional = vectorizer.transform(emotion_data["test"]["text"])
Y_test_traditional = one_hot_encode(emotion_data["test"]["labels"], num_label)

In [9]:
# SVM
svm_model = LinearSVC()
svm_model.fit(X_train_traditional, np.argmax(Y_train_traditional, axis=1))
y_pred_svm = svm_model.predict(X_test_traditional)
svm_accuracy = accuracy_score(np.argmax(Y_test_traditional, axis=1), y_pred_svm)
svm_precision = precision_score(np.argmax(Y_test_traditional, axis=1), y_pred_svm, average='weighted')
svm_recall = recall_score(np.argmax(Y_test_traditional, axis=1), y_pred_svm, average='weighted')
svm_f1 = f1_score(np.argmax(Y_test_traditional, axis=1), y_pred_svm, average='weighted')

c:\Users\duynt\.conda\envs\csci373\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Users\duynt\.conda\envs\csci373\lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\duynt\.conda\envs\csci373\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
# Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_traditional, np.argmax(Y_train_traditional, axis=1))
y_pred_nb = nb_model.predict(X_test_traditional)
nb_accuracy = accuracy_score(np.argmax(Y_test_traditional, axis=1), y_pred_nb)
nb_precision = precision_score(np.argmax(Y_test_traditional, axis=1), y_pred_nb, average='weighted')
nb_recall = recall_score(np.argmax(Y_test_traditional, axis=1), y_pred_nb, average='weighted')
nb_f1 = f1_score(np.argmax(Y_test_traditional, axis=1), y_pred_nb, average='weighted')

c:\Users\duynt\.conda\envs\csci373\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [11]:
# Hyperparameters for Transformer model
num_layers = 2
d_model = 256
num_heads = 8
dff = 512
input_vocab_size = num_words + 1

# Initialize and compile Transformer model
transformer_model = EncoderCategoryTransformer(num_layers, d_model, num_heads, dff, input_vocab_size, max_length, num_label)
optimizer = tf.keras.optimizers.Adam(0.0001)
transformer_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train Transformer model
history = transformer_model.fit(
    X_train_transformer, 
    Y_train_transformer, 
    batch_size=32, 
    epochs=4, 
    validation_data=(X_test_transformer, Y_test_transformer), 
    callbacks=[CSVLogger('results/go_emotions_log_0.0001.csv')]
)

Epoch 1/4
1135/1135 [==============================] - 71s 59ms/step - loss: 2.1154 - accuracy: 0.4821 - val_loss: 1.7568 - val_accuracy: 0.5580
Epoch 2/4
1135/1135 [==============================] - 67s 59ms/step - loss: 1.6262 - accuracy: 0.5725 - val_loss: 1.5968 - val_accuracy: 0.5830
Epoch 3/4
1135/1135 [==============================] - 68s 60ms/step - loss: 1.4331 - accuracy: 0.6002 - val_loss: 1.5805 - val_accuracy: 0.5760
Epoch 4/4
144/144 [==============================] - 3s 22ms/step


c:\Users\duynt\.conda\envs\csci373\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
# Evaluate Transformer model
y_pred_transformer = np.argmax(transformer_model.predict(X_test_transformer), axis=1)
transformer_accuracy = accuracy_score(np.argmax(Y_test_transformer, axis=1), y_pred_transformer)
transformer_precision = precision_score(np.argmax(Y_test_transformer, axis=1), y_pred_transformer, average='weighted')
transformer_recall = recall_score(np.argmax(Y_test_transformer, axis=1), y_pred_transformer, average='weighted')
transformer_f1 = f1_score(np.argmax(Y_test_transformer, axis=1), y_pred_transformer, average='weighted')

# Save the results to a csv file
results.to_csv('results/go_emotions_results_0.0001.csv', index=False)

results

144/144 [==============================] - 4s 23ms/step


c:\Users\duynt\.conda\envs\csci373\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,Model,Accuracy,Precision,Recall,F1
0,SVM,0.516776,0.489653,0.516776,0.497691
1,Naive Bayes,0.466449,0.490745,0.466449,0.378086
2,Transformer,0.563617,0.551570,0.563617,0.545892


c:\Users\duynt\.conda\envs\csci373\lib\site-packages\plotnine\ggplot.py:606: PlotnineWarning: Saving 10 x 5 in image.
c:\Users\duynt\.conda\envs\csci373\lib\site-packages\plotnine\ggplot.py:607: PlotnineWarning: Filename: results/go_emotions_results_0.0001.png


In [15]:
# Output the results with plotnine
from plotnine import *

results_long = results.melt(id_vars='Model', var_name='Metric', value_name='Score')

p = (
    ggplot(results_long, aes(x='Model', y='Score', fill='Model')) +
    geom_bar(stat='identity', position='dodge') +
    facet_wrap('~Metric', scales='free_y') +
    theme_minimal() +
    theme(legend_position='none')
)

p.save('results/go_emotions_results_0.0001.png', width=10, height=5, dpi=300)

p

c:\Users\duynt\.conda\envs\csci373\lib\site-packages\plotnine\ggplot.py:606: PlotnineWarning: Saving 10 x 5 in image.
c:\Users\duynt\.conda\envs\csci373\lib\site-packages\plotnine\ggplot.py:607: PlotnineWarning: Filename: results/go_emotions_history_0.0001.png


In [16]:
# Plot the training history of the Transformer model
history_df = pd.read_csv('results/go_emotions_log_0.0001.csv')

history_df_long = history_df.melt(id_vars='epoch', var_name='Metric', value_name='Score')

p2 = (
    ggplot(history_df_long, aes(x='epoch', y='Score', color='Metric')) +
    geom_line() +
    theme_minimal()
)

p2.save('results/go_emotions_history_0.0001.png', width=10, height=5, dpi=300)

p2